In [1]:
import sympy as sp
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Latex
from pint import UnitRegistry
ureg = UnitRegistry()
from CoolProp import CoolProp as cp
import scipy.constants as phys_c 
import ht, fluids

In [2]:
R, p, v, a, Cp, x, θ, T, h, k,ρ = sp .symbols('R p v a Cp x θ T h k ρ')

In [3]:
R = 1*ureg.mm
T_i = (273+327)*ureg.degK
T_a1 = (20+273)*ureg.degK
S = 4*np.pi*R**2


Propiedades físicas del Plomo: 
 Emisividad $\varepsilon=0.1$, densidad $\rho_{Pb}=10.5\cdot densidaddensidad10^3$kg/m$^3$, calor latente $h_{fg}=23$kJ/kg K, conductividad térmica $\kappa_{Pb}=16$ watt/m K, difusividad térmica $a_{Pb}=1\cdot 10^{-5}$m$^2$/s.


In [41]:
g = phys_c.g*ureg.meter/ureg.second**2

rho_pb  = 10.5e3*ureg.kilogram/ureg.meter**3
em_pb = 0.1
hfg_pb = 23e3*ureg.joule/ureg.kilogram
k_pb = 16*ureg.watt/ureg.meter/ureg.kelvin
a_pb = 1e-5 * ureg.meter**2/ureg.second
Tref = (T_i+T_a1)/2
rho_a = cp.PropsSI('D','T',Tref.magnitude,'P',101.3e3,'Air')
mu_a = cp.PropsSI('V','T',Tref.magnitude,'P',101.3e3,'Air')
k_a = cp.PropsSI('L','T',Tref.magnitude,'P',101.3e3,'Air')*ureg.watt/ureg.meter/ureg.kelvin
nu_a = mu_a / rho_a * ureg.meter**2/ureg.second
Prandtl_a = cp.PropsSI('Prandtl','T',Tref.magnitude,'P',101.3e3,'Air')
a_a = nu_a/Prandtl_a  
sigma = phys_c.sigma*ureg.watt/ureg.meter**2/ureg.kelvin**4
Q_rad1 = 1*sigma*T_i**4*S*em_pb

In [42]:
Q_rad1.to_base_units()

<Quantity(0.00923478103, 'kilogram * meter ** 2 / second ** 3')>

In [43]:
#Caso Espacio Exterior
#Calor total
Q_solid1 = 4/3*np.pi*R**3*hfg_pb*rho_pb
tiempo_1 = Q_solid1/Q_rad1
print(tiempo_1.to_base_units())
display(Latex(f'$Q_{{solid}}={Q_solid1.to_base_units():.2f}$'))

109.54161566956694 second


<IPython.core.display.Latex object>

In [44]:
#Caso Estacion espacial
# Q_cond = k_a*(T_i-T_a1)/long_dif
# long_dif = np.sqrt(a_a*tdif)
#Q_cond * tdif  = Q_solid1
tdif  = (Q_solid1/ (S*k_a*(T_i-T_a1)/np.sqrt(a_a)))**2
print(tdif.to_base_units())

2333.88856021108 second


In [45]:
long_dif = np.sqrt(a_a*tdif)
Q_cond = S*k_a*(T_i-T_a1)/long_dif
Q_cond*tdif*1e-9
(Q_cond).to_reduced_units()

<Quantity(0.000433436648, 'watt ** 2 * second / joule')>

In [46]:
Q_solid1.to_base_units(),(S*k_a*(T_i-T_a1)/np.sqrt(a_a)).to_base_units()

(<Quantity(1.01159283, 'meter ** 2 * kilogram / second ** 2')>,
 <Quantity(0.0209394701, 'meter ** 2 * kilogram / second ** 2.5')>)

In [50]:
Tm = (T_i+T_a1)/2
DeltaT = T_i-T_a1
Ra_D = R**3/Tm*DeltaT*g/nu_a/a_a
Nu_D = Ra_D**(1/4)
h_D = Nu_D*k_a/R
print(Ra_D.to_base_units())

4.7073931950504235 dimensionless


(<Quantity(293, 'kelvin')>, <Quantity(293, 'kelvin')>)

In [51]:
Q_conv_nat = h_D*S/2*DeltaT
t_conv_nat = Q_solid1/Q_conv_nat
print(t_conv_nat.to_base_units())

9.745625726299478 second


In [53]:
Q_conv_nat.to_base_units(),Ra_D

(<Quantity(0.10379968, 'meter ** 2 * kilogram / second ** 3')>,
 <Quantity(4.7073932e+09, 'millimeter ** 3 / meter ** 3')>)

In [23]:
Q_conv_nat.to_base_units(),Nu_D.to_base_units()

(<Quantity(0.11854994283822572, 'meter ** 2 * kilogram / second ** 3')>,
 <Quantity(2.3763532308927124, 'dimensionless')>)

In [24]:
from scipy.optimize import  brentq


def Ulim(U):
    R1 = R.to_base_units().magnitude
    Area_e = 4*np.pi*R1**2
    Vol_e = 4/3*np.pi*R1**3
    P = Vol_e*rho_pb.to_base_units().magnitude*g.to_base_units().magnitude
    Re = U*R1*2/nu_a.to_base_units().magnitude
    CD = fluids.drag_sphere(Re)
    FD = CD*0.5*rho_a*U**2*Area_e
    return FD-P
def Nu_sp(Re,Pr):
    Nu_D = 2+(0.4*Re**0.5+0.06*Re**(2/3))*Pr**0.4
    return Nu_D

In [32]:
fluids.drag_sphere(3000)

0.395884018776819

In [25]:
U = brentq(Ulim,0.1,30)
R1 = R.to_base_units().magnitude
Area_e = 4*np.pi*R1**2
Vol_e = 4/3*np.pi*R1**3
Re = U*R1*2/nu_a.to_base_units().magnitude
CD = fluids.drag_sphere(Re)
FD = CD*0.5*rho_a*U**2*Area_e
Nu_D = Nu_sp(Re,Prandtl_a)
h_sp = Nu_D*k_a.to_base_units().magnitude/(R1*2)

Q_conv_forced = h_sp*S.to_base_units().magnitude*DeltaT.to_base_units().magnitude

t = Q_solid1.to_base_units().magnitude/Q_conv_forced
display(Latex(f'time: {t:.2f}sec'))

<IPython.core.display.Latex object>

In [26]:
Vol_e*rho_pb.to_base_units().magnitude*g.to_base_units().magnitude,CD*0.5*rho_a*28**2*Area_e



(0.0004313189943485688, 0.002558711154874779)

In [27]:
2+(0.4*Re**0.5+0.06*Re**2/3)*Prandtl_a**0.4

40382.443598757505

In [28]:
FD,Vol_e*rho_pb.to_base_units().magnitude*g.to_base_units().magnitude,CD*0.5*rho_a*30**2*Area_e

(0.00043131899434855416, 0.0004313189943485688, 0.002937295968606252)

In [29]:
Q_rad1.to_base_units()

<Quantity(0.009234781030685089, 'kilogram * meter ** 2 / second ** 3')>

In [39]:
Q_conv_forced

1.1228855638739907

In [17]:
1/3.4e-3

294.11764705882354

# Radiacion Colector Solar

In [13]:
#Daros 
# Irradiacion Pcia Buenos Aires https://sig.energia.gob.ar/visor/visorsig.php
I = 6e3  #kwh/día
Q = I*ureg.watt/12
q = Q/ureg.meter**2

S = 2*ureg.meter**2
Tref = (100+273)*ureg.kelvin
Cp_agua = cp.PropsSI('C','T',Tref.magnitude,'P',101.3e3,'Water')*ureg.joule/ureg.kelvin/ureg.kilogram
rho_agua = cp.PropsSI('D','T',Tref.magnitude,'P',101.3e3,'Water')*ureg.kilogram/ureg.meter**3
Ti_agua = (20+273)*ureg.kelvin
Ts_agua = (80+273)*ureg.kelvin
G_agua = S*q/(Ts_agua-Ti_agua)/Cp_agua/rho_agua

L = 1*ureg.centimeter
Tamb = (30+273)*ureg.kelvin

In [14]:
q*S

1000.0 <Unit('watt')>

In [15]:
G_agua.to(ureg.liter/ureg.hour)*12*ureg.hour

178.201260055969 <Unit('liter')>

In [16]:
Tm_agua = (Ti_agua+Ts_agua)/2

In [17]:
Ra = L**3*1/Tm_agua*(Ts_agua-Tamb)*g/nu_a/a_a

In [18]:
h_c = Ra.to_base_units()**(1/4)*k_a/L
h_c.to(ureg.watt/ureg.meter**2/ureg.kelvin)*S*(Ts_agua-Tamb)

2142.679856495361 <Unit('watt')>

In [19]:
Ra.to_base_units()

4711.299490251705 <Unit('dimensionless')>

In [20]:
h_c.to_base_units()

21.42679856495361 <Unit('kilogram / kelvin / second ** 3')>